In [1]:
"""Minimal jobshop example."""
import collections
from ortools.sat.python import cp_model

In [72]:
# Data.
task_data = [  # (task_id, machine_id, processing_time)
    (0, 0, 3), (0, 1, 2), (0, 2, 2),  # Task0
    (1, 0, 2), (1, 1, 1), (1, 2, 2),  # Task1
    (2, 0, 6), (2, 1, 1),  # Task2
    (3, 0, 1) # Task3
]

# Encode job data

# Create CP model

In [73]:
# Create the model.

class UnrelatedMachinesScheduling:
    def __init__(self, task_data):
        self.task_data = task_data
        self.__define_model()
        self.__define_constraints()
        self.__define_objective()

    def __define_model(self):
        self.model = cp_model.CpModel()
        # Named tuple to store information about created variables.
        self.task_type = collections.namedtuple("task_type", "start end interval assigned")
        # Creates job intervals and add to the corresponding machine lists.
        self.all_tasks = {}
        self.task_assigned_machines = collections.defaultdict(list)
        self.machine_to_intervals = collections.defaultdict(list)
        self.intervals = collections.defaultdict(list)
        self.goal_variables = collections.defaultdict(list)
        self.horizon = sum(task[2] for task in self.task_data)
    
    
    def __define_constraints(self):
        #Define decision variable
        for task, machine, processing_time in self.task_data:
            suffix = f"_{task}_{machine}"
            assigned_var = self.model.NewIntVar(0, 1, 'assigned' + suffix)
            self.task_assigned_machines[task].append(assigned_var)
            start_var = self.model.NewIntVar(0, self.horizon, 'start' + suffix)
            end_var = self.model.NewIntVar(0, self.horizon, 'end' + suffix)
            interval_var = self.model.NewIntervalVar(
                start    = start_var,
                end      = end_var,
                size = processing_time,
                name = 'interval'+suffix
            )
            goal_variable = self.model.NewIntVar(0, self.horizon, 'goal' + suffix)
            self.model.AddMultiplicationEquality(goal_variable, assigned_var, end_var) 
            self.goal_variables[task, machine] = goal_variable
            self.intervals[task, machine] = self.task_type(
                start = start_var,
                end = end_var,
                assigned = assigned_var,
                interval = interval_var
            )
            self.machine_to_intervals[machine].append(interval_var)

        #Constraint 1: No machine may work on more than one task simultaneously
        machines_count = 1 + max(task[1] for task in self.task_data)
        all_machines = range(machines_count)
        for machine in all_machines:
            #no overlap between intervals of one machine
            self.model.AddNoOverlap(self.machine_to_intervals[machine])

        #Constraint 2: All tasks must be assigned to exactly one machine
        tasks_count = 1+ max(task[0] for task in self.task_data)
        all_tasks = range(tasks_count)
        for task in all_tasks:
            #sum of assignment variable of task must be one
            self.model.Add(cp_model.LinearExpr.Sum(self.task_assigned_machines[task]) == 1)
        
    def __define_objective(self):
        # Makespan objective.
        obj_var = self.model.NewIntVar(0, self.horizon, "makespan")
        self.model.AddMaxEquality(
            obj_var,
            [self.goal_variables[task, machine] for task, machine, _ in self.task_data],
        )
        self.model.Minimize(obj_var)

    def solve(self, solver=cp_model.CpSolver()):
        status = solver.Solve(self.model)
        return (solver, status)


model = UnrelatedMachinesScheduling(task_data)

In [74]:
# Creates the solver and solve.
solver, status = model.solve()

print('feasible:', status == cp_model.FEASIBLE, ', optimal:', status == cp_model.OPTIMAL)

feasible: False , optimal: True


In [75]:
for iid, interval in model.intervals.items():
    if solver.Value(interval.assigned):
        print(iid, f"{solver.Value(interval.start)}-{solver.Value(interval.end)}")

(0, 2) 0-2
(1, 1) 0-1
(2, 1) 1-2
(3, 0) 0-1
